💡 **LUMINA** 💡

Sistema inteligente de detecção de golpes, projetado para proteger usuários, especialmente idosos.

Instala a biblioteca google-genai

In [1]:
%pip -q install google-genai

Configura a chave da API do Google Gemini.

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

Inicializa o cliente da SDK do Gemini e define o ID do modelo.

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

Instala o framework google-adk para agentes.

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

Importa as bibliotecas necessárias para criar e executar os agentes, manipular dados, formatar texto e exibir resultados.

In [13]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

Define a função call_agent para interagir com os agentes, enviando mensagens e obtendo respostas. A lógica está funcional para comunicação com os agentes.

In [65]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

Define a função to_markdown para formatar o texto para exibição no Colab usando Markdown.

In [66]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Testa a chamada ao modelo Gemini com a ferramenta de busca e exibe a resposta. A chamada está funcionando corretamente.

In [16]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents='',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Okay, I'm ready. What is your question?


Define o agente_triagem para coletar e processar a mensagem do usuário, buscando informações sobre golpes.

In [92]:
# 👮 Agente de Triagem

def agente_triagem(mensagem_usuario, hoje):
      triagem = Agent(
          name="agente_triagem",
          model="gemini-2.0-flash",
          instruction="""
          Você é um agente de triagem de segurança digital. Sua função é apenas coletar e processar a mensagem recebida de forma estratégica.

          Instruções:

              Não escreva uma resposta direta para o usuário final.

              Faça uma pesquisa com a ferramenta (google_search) sobre os principais tipos de golpes aplicados nos últimos 5 anos contra a
              população brasileira (e principalmente com idosos).

              A partir da mensagem recebida, compare com os sinais de golpe encontrados e colete apenas ações emergenciais que a pessoa deveria tomar
              (ex: bloquear, não clicar, não responder, confirmar com familiar).

              Seja objetivo. Sua resposta servirá como insumo para outro agente analista.

          """,
          tools=[google_search],
          description="Agente de triagem"
      )

      resultados_triagem = f"Mensagem: {mensagem_usuario}\nData de hoje: {hoje}"
      lancamentos = call_agent(triagem, resultados_triagem)
      return lancamentos


Define o agente_financeiro para analisar a mensagem e o resultado da triagem, verificando a autenticidade da mensagem.

In [101]:
# 👮 Agente Analista Financeiro

def agente_financeiro(mensagem_usuario, resultado_triagem):
      financeiro = Agent(
          name="agente_financeiro",
          model="gemini-2.0-flash",
          instruction="""

          Você é um analista de segurança financeira digital. Receberá duas informações:

            Uma síntese das ações emergenciais sugeridas pela triagem.

            A mensagem suspeita recebida pelo usuário.

          Instruções:

              Verifique se o conteúdo da mensagem está alinhado com o comportamento e práticas de órgãos oficiais, bancos, institutos de previdência, seguradoras ou instituições governamentais.

              Investigue usando (google_search), se necessário.

              Se identificar incompatibilidades, aponte-as claramente. Se for legítimo, justifique também.

              Escreva de forma clara, confiável e sem jargões. Lembre-se: sua análise será lida por um agente posterior e pode impactar uma decisão de risco.

          """,
          tools=[google_search],
          description="Agente analista financeiro."
      )

      resultados_financeiro = f"Mensagem: {mensagem_usuario}\nResultado da triagem: {resultado_triagem}"
      return resultados_financeiro


Define o agente_risco para avaliar o risco da situação e fornecer um nível de risco, motivo e ação recomendada.

In [98]:
# 👮 Agente de Avaliação de Risco

def agente_risco(analise_financeira):
    risco = Agent(
        name="agente_risco",
        model="gemini-2.0-flash",
        instruction="""
        Sua tarefa:
        Com base na análise financeira, forneça apenas três informações separadas por linhas:

          1. Nível de risco (número inteiro entre 1 e 4)
          2. Motivo do risco em até 2 frases
          3. Ação recomendada para o usuário (clara e direta)

        Não use emojis. Não escreva introdução nem conclusão. Apenas as três respostas, cada uma em uma linha.

        """,
        description="Agente analista de riscos"
    )

    resultado_risco = call_agent(risco, analise_financeira)
    return resultado_risco

Executa o sistema principal, coletando a mensagem do usuário, chamando os agentes em sequência e exibindo o resultado final. O fluxo principal do sistema está implementado e funcional.

In [103]:
# 🧠 Execução do Sistema

hoje = date.today().strftime("%d/%m/%Y")
print("🔐 Bem-vindo ao LUMINA 💡\nUm sistema de detecção de golpes sofisticado que poderá te ajudar em poucos minutos!")
mensagem_usuario = input("📨 Cole, digite a mensagem (SMS, e-mail, etc.) ou nos conte a história de que você esteja suspeitando ser um golpe:\n").strip()

if not mensagem_usuario:
    print("⚠️ Você esqueceu de digitar a mensagem.")
else:
    print("\n✅ Recebemos sua mensagem. Aguarde um minutinho enquanto verificamos...\n")

    print("🔎 Etapa 1: Analisando caso...\n")
    coleta_triagem = agente_triagem(mensagem_usuario, hoje)
    display(to_markdown("### 🔍 Ações Emergenciais Coletadas pela Triagem:\n" + coleta_triagem))

    print("\n💸 Etapa 2: Verificação de comportamento de instituições oficiais...\n")
    analise_financeira = agente_financeiro(mensagem_usuario, coleta_triagem)
    display(to_markdown("### 🧾 Análise Institucional Financeira:\n" + analise_financeira))

    print("\n📊 Etapa 3: Avaliação de Risco Final...\n")
    resultado_bruto = agente_risco(analise_financeira)

    # Divide a resposta em partes
    linhas = resultado_bruto.split("\n")
    try:
        nivel = int(linhas[0].strip())
        motivo = linhas[1].strip() if len(linhas) > 1 else "Motivo não informado."
        acao = linhas[2].strip() if len(linhas) > 2 else "Ação não informada."
    except (ValueError, IndexError):
        nivel = -1
        motivo = "Não foi possível interpretar o nível de risco."
        acao = "Revise a análise."

    # Mapeamento de emojis
    emojis = {
        1: "🟢",
        2: "🟡🟡",
        3: "🔴🔴🔴",
        4: "🔴🔴🔴🔴"
    }
    emoji_nivel = emojis.get(nivel, "❓")

    # Exibir resultado
    resultado_formatado = f"""
    ### 🔐 Classificação de Risco
    Nível de Risco: {nivel} {emoji_nivel}

    🧭 Motivo:
    {motivo}

    🛡️ Ação Recomendada:
    {acao}
    """

    display(to_markdown(resultado_formatado))

🔐 Bem-vindo ao LUMINA 💡
Um sistema de detecção de golpes sofisticado que poderá te ajudar em poucos minutos!
📨 Cole, digite a mensagem (SMS, e-mail, etc.) ou nos conte a história de que você esteja suspeitando ser um golpe:
oi aqui eh o itau

✅ Recebemos sua mensagem. Aguarde um minutinho enquanto verificamos...

🔎 Etapa 1: Analisando caso...



> ### 🔍 Ações Emergenciais Coletadas pela Triagem:
> Com base na mensagem recebida ("oi aqui eh o itau") e nos golpes mais comuns no Brasil, especialmente aqueles que visam idosos, as seguintes ações emergenciais devem ser consideradas:
> 
> *   **Desconfie da mensagem:** Bancos geralmente não iniciam contato dessa forma informal.
> 
> *   **Não responda à mensagem:** Evite qualquer interação adicional.
> 
> *   **Não clique em links:** Se houver links na mensagem, não clique neles.
> 
> *   **Entre em contato com o Itaú por canais oficiais:** Ignore a mensagem e entre em contato diretamente com o banco através dos canais de atendimento conhecidos (telefone, aplicativo oficial, agência).
> 
> *   **Confirme com um familiar ou pessoa de confiança:** Consulte alguém próximo antes de tomar qualquer atitude.
> 
> *   **Considere bloquear o número:** Bloquear o número remetente pode evitar novas tentativas de contato.
> 



💸 Etapa 2: Verificação de comportamento de instituições oficiais...



> ### 🧾 Análise Institucional Financeira:
> Mensagem: oi aqui eh o itau
> Resultado da triagem: Com base na mensagem recebida ("oi aqui eh o itau") e nos golpes mais comuns no Brasil, especialmente aqueles que visam idosos, as seguintes ações emergenciais devem ser consideradas:
> 
> *   **Desconfie da mensagem:** Bancos geralmente não iniciam contato dessa forma informal.
> 
> *   **Não responda à mensagem:** Evite qualquer interação adicional.
> 
> *   **Não clique em links:** Se houver links na mensagem, não clique neles.
> 
> *   **Entre em contato com o Itaú por canais oficiais:** Ignore a mensagem e entre em contato diretamente com o banco através dos canais de atendimento conhecidos (telefone, aplicativo oficial, agência).
> 
> *   **Confirme com um familiar ou pessoa de confiança:** Consulte alguém próximo antes de tomar qualquer atitude.
> 
> *   **Considere bloquear o número:** Bloquear o número remetente pode evitar novas tentativas de contato.
> 



📊 Etapa 3: Avaliação de Risco Final...



> 
>     ### 🔐 Classificação de Risco
>     Nível de Risco: 4 🔴🔴🔴🔴
> 
>     🧭 Motivo:
>     A mensagem não é formal e solicita informações pessoais, o que pode indicar uma tentativa de golpe.
> 
>     🛡️ Ação Recomendada:
>     Entre em contato com o Itaú por canais oficiais para verificar a autenticidade da mensagem.
>     